In [1]:
import pandas as pd
import networkx as nx

### Lettura dataset e selezione dei soli giocatori in entrata (evitando così doppi trasferimenti e giocatori svincolati)

In [2]:
df = pd.read_csv('./dataset/dataset/transfers.csv')
dataset = df[df['dir'] == 'in']
dataset

,league,season,window,team_id,team_name,team_country,dir,player_id,player_name,player_age,...,counter_team_id,counter_team_name,counter_team_country,transfer_fee_amnt,market_val_amnt,is_free,is_loan,is_loan_end,is_retired,transfer_id
0,GB1,2009,s,985,Manchester United,England,in,33544,Antonio Valencia,23.0,...,1071,Wigan Athletic,England,18800000.0,NaN,False,False,False,False,310832
1,GB1,2009,s,985,Manchester United,England,in,62049,Mame Diouf,21.0,...,687,Molde FK,Norway,4500000.0,1600000.0,False,False,False,False,319841
2,GB1,2009,s,985,Manchester United,England,in,43261,Gabriel Obertan,20.0,...,40,FC Girondins Bordeaux,France,4000000.0,400000.0,False,False,False,False,315185
3,GB1,2009,s,985,Manchester United,England,in,1397,Michael Owen,29.0,...,762,Newcastle United,England,0.0,NaN,True,False,False,False,306421
4,GB1,2009,s,985,Manchester United,England,in,73538,Scott Moffatt,18.0,...,5242,Manchester United U18,England,NaN,NaN,False,False,False,False,339015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70000,NL1,2021,w,385,Fortuna Sittard,Netherlands,in,140032,Andreas Samaris,32.0,...,515,Without Club,Without Club,NaN,1500000.0,False,False,False,False,3633188
70001,NL1,2021,w,306,SC Heerenveen,Netherlands,in,257808,Runar Espejord,25.0,...,1293,Tromsø IL,Norway,NaN,500000.0,False,True,True,False,3071862
70002,NL1,2021,w,306,SC Heerenveen,Netherlands,in,580142,Joaquín Fernández,22.0,...,37535,Montevideo City Torque,Uruguay,NaN,100000.0,False,True,True,False,3268245
70003,NL1,2021,w,468,Sparta Rotterdam,Netherlands,in,340353,Maduka Okoye,22.0,...,1010,Watford FC,England,NaN,1000000.0,False,True,False,False,3619917


In [3]:
print('colonna player_age Nan -> ' , dataset['player_age'].isna().sum())
print('colonna transfer_fee_amnt Nan -> ' , dataset['transfer_fee_amnt'].isna().sum())
print('colonna market_val_amnt Nan -> ' , dataset['market_val_amnt'].isna().sum())
print('colonna team_name Nan -> ' , dataset['team_name'].isna().sum())
print('colonna counter_team_name Nan -> ' , dataset['counter_team_name'].isna().sum())

colonna player_age Nan ->  6
colonna transfer_fee_amnt Nan ->  18669
colonna market_val_amnt Nan ->  7523
colonna team_name Nan ->  0
colonna counter_team_name Nan ->  0


### Preprocessing:
##### - Rimuoviamo i 6 trasferimenti il cui player_age è Nan (per eventuali analisi future sull'età)
##### - Sostituiamo i Nan di transfer_fee_amnt con 0 (per eventuali analisi future sui valori di trasferimento)
##### - Sostituiamo i Nan di market_val_amnt con 0 (per eventuali analisi future sui valori di mercato)
##### - Seleziono solo i trasferimenti in cui la colonna 'counter_team_name' è diversa da 'Without Club' in quanto questa indica giocatori svincolati
##### - Seleziono solo i trasferimenti in cui la colonna 'team_name' è diversa da 'Without Club' in quanto questa indica l'ingaggio di giocatori svincolati


In [4]:
# rimuovi i 6 player che non hanno età
dataset.dropna(subset=['player_age'], inplace=True)
# sostituisci i valori di trasferimento Nan con 0
dataset['transfer_fee_amnt'].fillna(0, inplace=True)
# sostituisci i valori di valore di mercato Nan con 0
dataset['market_val_amnt'].fillna(0, inplace=True)
# rimuovi i trasferimenti senza squadra di destinazione (ovvero svincoli)
dataset = dataset[dataset['counter_team_name'] != 'Without Club'] 
# rimuovi i trasferimenti senza squadra di provenienza (ovvero ingaggio di svincolati)
dataset = dataset[dataset['team_name'] != 'Without Club']
print('Il dataset è composto da ', dataset.shape[0], ' trasferimenti')

Il dataset è composto da  31416  trasferimenti


/var/folders/61/9rphml817c1d45n8g26p1ltm0000gn/T/ipykernel_937/2582955158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.dropna(subset=['player_age'], inplace=True)
/var/folders/61/9rphml817c1d45n8g26p1ltm0000gn/T/ipykernel_937/2582955158.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['transfer_fee_amnt'].fillna(0, inplace=True)
/var/folders/61/9rphml817c1d45n8g26p1ltm0000gn/T/ipykernel_937/2582955158.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

### Inizio costruzione della rete
#### Ogni team deve avere:
##### - un id
##### - un nome
##### - un paese e un codice lega (GB1 ad esempio)
### I team devono essere presi sia da team_name che da counter_team_name
#### Ogni trasferimento deve avere: 
##### - un id di trasferimento (?)
##### - un player id 
##### - un player name
##### - l'età del player
##### - una season
##### - una finestra di mercato (invernale o estiva)
##### - un valore di trasferimento
##### - un bool che indica se è in prestito o a titolo definitivo
##### - il valore di mercato (?)

In [28]:
# Crea un grafo diretto
G = nx.DiGraph()

# Seleziona le squadre che acquistano il giocatore  
teams_in = dataset[['team_id', 'team_name', 'team_country']]

# Seleziona le squadre che vendono il giocatore
teams_out = dataset[['counter_team_id', 'counter_team_name', 'counter_team_country']]
teams_out = teams_out.rename(columns={'counter_team_id': 'team_id'})
teams_out = teams_out.rename(columns={'counter_team_name': 'team_name'})
teams_out = teams_out.rename(columns={'counter_team_country': 'team_country'})

# Concatena i DataFrame in un unico DataFrame
teams = pd.concat([teams_in, teams_out])

# Rimuovi le righe duplicate
teams = teams.drop_duplicates()

for index, row in teams.iterrows():
    # il primo è l'identficativo del nodo, il secondo è una label che etichetta il tipo di nodo, gli altri sono attributi
    G.add_node(row['team_id'], label='team', name=row['team_name'], country=row['team_country'])


# Aggiungi gli archi che rappresentano i trasferimenti tra squadre 
for index, row in dataset.iterrows():
    transfer_attributes = {
        'transfer_id': row['transfer_id'],
        'player_id': row['player_id'],
        'player_name': row['player_name'],
        'player_age': row['player_age'],
        'season': row['season'],
        'window': row['window'],
        'transfer_fee': row['transfer_fee_amnt'],
        'is_loan': row['is_loan'],
        'market_value': row['market_val_amnt']
    }

    # Aggiungi gli archi che rappresentano i trasferimenti tra squadre (essendo solo trasferimenti in ingresso si aggiungono solo archi da squadra venditrice (counter_team) a squadra (team))
    G.add_edge(row['counter_team_id'], row['team_id'],  # Inverti l'ordine dei nodi
               label='transfer',
               **transfer_attributes)  # Aggiungi gli attributi del trasferimento

In [29]:
print('Nodi -> ', G.number_of_nodes())
print('Archi -> ', G.number_of_edges())
print('Density -> ', nx.density(G))

Nodi ->  2726
Archi ->  17352
Density ->  0.0023359157821050435
